**MCMC(j)- TCI(k,l)**

In [ ]:
import TensorCrossInterpolation as TCI

function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    NSq = N^2
    
    term1 = cos(2*π*(j*k/NSq)) * sin(2*π*(j*l/NSq)) * exp(-j/N)
    term2 = sin(2*π*(j*k/NSq)) * cos(2*π*(j*l/NSq)) * exp(-j^2/NSq)

    return term1 + term2
end


function get_tci(j)
    localdims = fill(N, 2)
    g(v) = f([j, v[1], v[2]]) #the method of fix one variable
    
    tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = 1e-8 )
    
    mps = tci_2d.sitetensors
    
    sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
    sumL = dropdims(sum(mps[2],dims=2),dims=2)

    resultMatrix = sumK * sumL #1x1 matrix
    
    return resultMatrix[1]
end


function mc_tci_sum(M)
    #define visit array
    vst = []

    #set init point
    j = rand(1:N)
    now = get_tci(j)
    memJ[j] = now
    push!(vst,now)
    

    for idx in 1:(M-1)
        jump = rand([-100,-10,-1,0,1,10,100])
        
        if (j + jump) < 1 || (j + jump) > N
            push!(vst,now)
            continue
        else
            j = j + jump

            #calculate proposal point
            prp = 0.0
            if memJ[j] == 0
                prp = get_tci(j)
                memJ[j] = prp
            else
                prp = memJ[j]
            end

            #MCMC
            accRatio = prp/now
            if rand() < accRatio
                #acc
                push!(vst,prp) 
                now = prp
            else
                #rej
                push!(vst,now) 
                j = j - jump #undo
            end
        end
    end

    return sum(vst)
end

N = 1_000 #Num of element in one axis
M = 100 #number of sampling
memJ = zeros(N)

epoch = 10
acc = 0
accSq = 0
for i in 1:epoch
    println("epoch = $i")
    @time mctciSum = mc_tci_sum(M)

    acc += mctciSum
    accSq += mctciSum^2
end

stdev = sqrt((accSq / epoch - (acc / epoch)^2)/epoch)

println("\nnumber of j : $M")
println("MCMC-TCI stdev : $stdev" )
println("MCMC-TCI result : ", acc/epoch)


epoch = 1
